In [1]:
import json
import os
from glob import glob
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import wandb

from IPython.display import display, HTML

In [30]:
api = wandb.Api()

In [31]:
# The W&B API also provides a way for you to query across runs in a project with api.runs(). The most common use case is exporting runs data for custom analysis.  The query interface is the same as the one MongoDB uses.
# wandb.init("analysis", mode="offline")
def download_test_runs(overwrite=False):
    runs = api.runs("ajayj/nerf-nl", {"$and": [{"config.render_only": True, "config.render_test": True, "state": "finished"}]},
                    order='-created_at')
    print("Found %i" % len(runs))

    # Get most recent run of each
    runs_by_expname = {}
    for run in runs:
        expname = run.config["expname"]
        if expname not in runs_by_expname:
            runs_by_expname[expname] = run
    print("Filtered to %i most recent and unique" % len(runs_by_expname))

    # Download metrics
    for run in runs_by_expname.values():
        #metrics_path = f'logs/{run.config["expname"]}/renderonly_test_*/test_metrics.json'
        files = run.files()
        for file in files:
            if file.name.startswith(f'logs/{run.config["expname"]}/renderonly_test_') and file.name.endswith('/test_metrics.json'):
                if not overwrite and os.path.exists(file.name):
                    continue
                print('Downloading', file.name)

                try:
                    file.download(root='.', replace=True)
                except Exception as e:
                    print('download error', file.name, e)

    return runs_by_expname

In [32]:
runs_by_expname = download_test_runs()

Found 163
Filtered to 68 most recent and unique


In [33]:
# config_blacklist = [
#     'configs/257_blender_materials_8views_ctr.txt',
# ]

expname_blacklist = [
    '257_blender_materials_8views_ctr',
    '287_blender_materials_8views_ftctr258_150k',
    '288_blender_hotdog_8views_ftctr256_150k',
    '256_blender_hotdog_8views_ctr'  # use 256_blender_hotdog_8views_ctr_resume150k instead
]

In [34]:
# Load metrics into a dataframe
# print("Skipping", set(glob('logs/*/renderonly_test_*/test_metrics.json')) - set(glob('logs/*/renderonly_test_199999/test_metrics.json')))

metrics_list = []
for metrics_path in glob('logs/*/renderonly_test_*/test_metrics.json'):
    with open(metrics_path, 'r') as metrics_f:
        metrics_and_args = json.load(metrics_f)
        
        # extract iteration count
        # metrics_and_args['test_metrics_path'] = metrics_path
        m = re.match(r'logs/.+/renderonly_test_([0-9]+)/test_metrics.json', metrics_path)
        assert m
        metrics_and_args['train_iters'] = int(m.group(1))
        # extract training view count
        metrics_and_args['views'] = metrics_and_args["args"]['max_train_views']
        metrics_and_args['views'] = metrics_and_args['views'] if metrics_and_args['views'] >= 0 else 100

        if metrics_and_args['args']['consistency_loss'] != 'none':
            metrics_and_args['method'] = 'SCaRF (ours)'
        elif 'ftctr' in metrics_and_args['args']['expname']:
            metrics_and_args['method'] = 'SCaRF, finetuned (ours)'
        elif any(simple_key in metrics_and_args['args']['expname'] for simple_key in ['simple', 'simplest', 'simpler', 'tune']):
            metrics_and_args['method'] = 'Simplified NeRF'
        else:
            metrics_and_args['method'] = 'NeRF'        

        # add columns for arguments
        for k, v in metrics_and_args["args"].items():
            metrics_and_args[f"args.{k}"] = v
        del metrics_and_args["args"]

        if metrics_and_args['args.expname'] in expname_blacklist:
            print('Skipping', metrics_and_args['args.config'], '(blacklist)')
            continue

#         if 'fullres' in metrics_and_args['args.expname']:
#             print(metrics_and_args)
        assert metrics_and_args['args.half_res'] == True
    
        metrics_list.append(metrics_and_args)

metrics_df = pd.DataFrame(metrics_list)

Skipping configs/288_blender_hotdog_8views_ftctr.txt (blacklist)
Skipping configs/288_blender_hotdog_8views_ftctr.txt (blacklist)
Skipping configs/288_blender_hotdog_8views_ftctr.txt (blacklist)
Skipping configs/288_blender_hotdog_8views_ftctr.txt (blacklist)
Skipping configs/257_blender_materials_8views_ctr.txt (blacklist)
Skipping configs/257_blender_materials_8views_ctr.txt (blacklist)
Skipping configs/287_blender_materials_8views_ftctr.txt (blacklist)
Skipping configs/287_blender_materials_8views_ftctr.txt (blacklist)
Skipping configs/287_blender_materials_8views_ftctr.txt (blacklist)
Skipping configs/287_blender_materials_8views_ftctr.txt (blacklist)
Skipping configs/256_blender_hotdog_8views_ctr.txt (blacklist)
Skipping configs/256_blender_hotdog_8views_ctr.txt (blacklist)


In [35]:
metrics_df[metrics_df['args.expname'] == '256_blender_hotdog_8views_ctr_resume']

,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.reload_iter,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.hardcode_train_views,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp,args.num_render_poses


# Group by scene, for all iterations

In [36]:
for scene_key, scene_df in metrics_df.groupby('args.datadir'):
    print(scene_key)
    scene_df = scene_df.sort_values([
        'views',
        'train_iters'
    ])
    display(scene_df)
#     print('==='*20)

./data/nerf_synthetic/chair


,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.reload_iter,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.hardcode_train_views,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp,args.num_render_poses
51,0.052804,12.973991,0.601996,0.415411,199999,4,NeRF,configs/220_blender_chair_4views.txt,220_blender_chair_4views,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,0.017929,18.436800,0.799504,0.190794,199999,4,SCaRF (ours),configs/240_blender_chair_4views_ctr.txt,240_blender_chair_4views_ctr,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,0.004178,24.620086,0.897281,0.094988,159999,8,SCaRF (ours),configs/250_blender_chair_8views_ctr.txt,250_blender_chair_8views_ctr,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.007454,21.869533,0.859454,0.151596,199999,8,Simplified NeRF,configs/330_blender_chair_8views_simple.txt,330_blender_chair_8views_simple,./logs,...,None,False,200000.0,1.0,[],0.0,0.0,single_random,1.0,NaN
66,0.004170,24.633213,0.898032,0.092438,199999,8,SCaRF (ours),configs/250_blender_chair_8views_ctr.txt,250_blender_chair_8views_ctr,./logs,...,NaN,False,200000.0,10.0,NaN,0.0,0.0,single_random,1.0,NaN
83,0.004170,24.633213,0.898032,0.092438,199999,8,"SCaRF, finetuned (ours)",configs/283_blender_chair_8views_ftctr.txt,283_blender_chair_8views_ftctr250,./logs,...,NaN,False,400000.0,1.0,NaN,0.0,0.0,single_random,1.0,NaN
88,0.011285,21.048566,0.860601,0.162509,199999,8,NeRF,configs/230_blender_chair_8views.txt,230_blender_chair_8views,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85,0.003651,25.316919,0.908050,0.083128,209999,8,"SCaRF, finetuned (ours)",configs/283_blender_chair_8views_ftctr.txt,283_blender_chair_8views_ftctr250,./logs,...,210000,False,400000.0,1.0,NaN,0.0,0.0,single_random,1.0,NaN
82,0.003459,25.595321,0.912456,0.077383,249999,8,"SCaRF, finetuned (ours)",configs/283_blender_chair_8views_ftctr.txt,283_blender_chair_8views_ftctr250,./logs,...,250000,False,400000.0,1.0,NaN,0.0,0.0,single_random,1.0,NaN
86,0.003425,25.627192,0.913511,0.076536,299999,8,"SCaRF, finetuned (ours)",configs/283_blender_chair_8views_ftctr.txt,283_blender_chair_8views_ftctr250,./logs,...,300000,False,400000.0,1.0,NaN,0.0,0.0,single_random,1.0,NaN


./data/nerf_synthetic/drums


,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.reload_iter,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.hardcode_train_views,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp,args.num_render_poses
15,0.045604,13.457292,0.675309,0.301415,199999,4,SCaRF (ours),configs/241_blender_drums_4views_ctr.txt,241_blender_drums_4views_ctr,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,0.086626,10.671803,0.590800,0.423424,199999,4,NeRF,configs/221_blender_drums_4views.txt,221_blender_drums_4views,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,0.009931,20.154216,0.845756,0.115702,159999,8,SCaRF (ours),configs/251_blender_drums_8views_ctr.txt,251_blender_drums_8views_ctr,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.018335,17.471725,0.769530,0.231024,199999,8,NeRF,configs/231_blender_drums_8views.txt,231_blender_drums_8views,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,0.010230,20.033635,0.843434,0.117413,199999,8,SCaRF (ours),configs/251_blender_drums_8views_ctr.txt,251_blender_drums_8views_ctr,./logs,...,NaN,False,200000.0,10.0,NaN,0.0,0.0,single_random,1.0,NaN
37,0.032963,15.020734,0.727292,0.280287,199999,8,Simplified NeRF,configs/331_blender_drums_8views_simple.txt,331_blender_drums_8views_simple,./logs,...,None,False,200000.0,1.0,[],0.0,0.0,single_random,1.0,NaN
71,0.010230,20.033635,0.843434,0.117413,199999,8,"SCaRF, finetuned (ours)",configs/284_blender_drums_8views_ftctr.txt,284_blender_drums_8views_ftctr251,./logs,...,NaN,False,400000.0,1.0,NaN,0.0,0.0,single_random,1.0,NaN
69,0.010116,20.081180,0.845624,0.115622,209999,8,"SCaRF, finetuned (ours)",configs/284_blender_drums_8views_ftctr.txt,284_blender_drums_8views_ftctr251,./logs,...,210000,False,400000.0,1.0,NaN,0.0,0.0,single_random,1.0,NaN
72,0.010237,20.029285,0.845375,0.117360,249999,8,"SCaRF, finetuned (ours)",configs/284_blender_drums_8views_ftctr.txt,284_blender_drums_8views_ftctr251,./logs,...,250000,False,400000.0,1.0,NaN,0.0,0.0,single_random,1.0,NaN
68,0.010379,19.975798,0.844611,0.117765,299999,8,"SCaRF, finetuned (ours)",configs/284_blender_drums_8views_ftctr.txt,284_blender_drums_8views_ftctr251,./logs,...,300000,False,400000.0,1.0,NaN,0.0,0.0,single_random,1.0,NaN


./data/nerf_synthetic/ficus


...

,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.reload_iter,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.hardcode_train_views,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp,args.num_render_poses
53,0.198314,7.085420,0.410553,0.524728,29999,4,SCaRF (ours),configs/247_blender_materials_4views_ctr.txt,247_blender_materials_4views_ctr,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,0.210504,6.808063,0.394229,0.532183,199999,4,SCaRF (ours),configs/247_blender_materials_4views_ctr.txt,247_blender_materials_4views_ctr,./logs,...,NaN,False,200000.0,10.0,NaN,0.0,0.0,single_random,1.0,NaN
90,0.165159,7.979751,0.497654,0.468705,199999,4,NeRF,configs/227_blender_materials_4views.txt,226_blender_materials_4views,./logs,...,NaN,False,200000.0,1.0,NaN,0.0,0.0,single_random,1.0,NaN
12,0.007130,21.699216,0.847500,0.112648,29999,8,SCaRF (ours),configs/258_blender_materials_8views_ctr.txt,258_blender_materials_8views_ctr,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,0.164257,7.869122,0.483181,0.470464,79999,8,NeRF,configs/237_blender_materials_8views.txt,237_blender_materials_8views,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,0.008147,21.253789,0.842860,0.102170,199999,8,SCaRF (ours),configs/258_blender_materials_8views_ctr.txt,258_blender_materials_8views_ctr,./logs,...,NaN,False,200000.0,10.0,NaN,0.0,0.0,single_random,1.0,NaN
24,0.165473,7.837306,0.483915,0.466599,199999,8,NeRF,configs/237_blender_materials_8views.txt,237_blender_materials_8views,./logs,...,NaN,False,200000.0,1.0,NaN,0.0,0.0,single_random,1.0,NaN
30,0.008937,20.659359,0.823098,0.150610,199999,8,Simplified NeRF,configs/347_blender_materials_8views_simpler.txt,347_blender_materials_8views_simpler,./logs,...,None,False,200000.0,1.0,[],0.0,0.0,single_random,1.0,NaN
26,0.007666,21.525626,0.848610,0.096937,209999,8,"SCaRF, finetuned (ours)",configs/287_blender_materials_8views_ftctr_200...,287_blender_materials_8views_ftctr258_200k,./logs,...,210000,False,400000.0,1.0,NaN,0.0,0.0,single_random,1.0,NaN
27,0.007490,21.620932,0.850885,0.094869,249999,8,"SCaRF, finetuned (ours)",configs/287_blender_materials_8views_ftctr_200...,287_blender_materials_8views_ftctr258_200k,./logs,...,250000,False,400000.0,1.0,[],0.0,0.0,single_random,1.0,40.0


./data/nerf_synthetic/mic


,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.reload_iter,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.hardcode_train_views,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp,args.num_render_poses
21,0.012596,19.239916,0.877721,0.120375,199999,4,SCaRF (ours),configs/244_blender_mic_4views_ctr.txt,244_blender_mic_4views_ctr,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,0.058747,12.372348,0.707625,0.346917,199999,4,NeRF,configs/224_blender_mic_4views.txt,224_blender_mic_4views,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,0.002806,26.277187,0.943431,0.054970,149999,8,SCaRF (ours),configs/254_blender_mic_8views_ctr.txt,254_blender_mic_8views_ctr,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,0.002838,26.321344,0.943849,0.052638,199999,8,SCaRF (ours),configs/254_blender_mic_8views_ctr.txt,254_blender_mic_8views_ctr,./logs,...,NaN,False,200000.0,10.0,NaN,0.0,0.0,single_random,1.0,NaN
29,0.003241,26.287436,0.944459,0.067203,199999,8,NeRF,configs/234_blender_mic_8views.txt,234_blender_mic_8views,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,0.004553,24.206465,0.929758,0.080002,199999,8,Simplified NeRF,configs/334_blender_mic_8views_simple.txt,334_blender_mic_8views_simple,./logs,...,None,False,200000.0,1.0,[],0.0,0.0,single_random,1.0,NaN
95,0.002838,26.321344,0.943849,0.052638,199999,8,"SCaRF, finetuned (ours)",configs/282_blender_mic_8views_ftctr.txt,282_blender_mic_8views_ftctr254,./logs,...,NaN,False,400000.0,1.0,NaN,0.0,0.0,single_random,1.0,NaN
92,0.002612,26.742503,0.947659,0.046870,209999,8,"SCaRF, finetuned (ours)",configs/282_blender_mic_8views_ftctr.txt,282_blender_mic_8views_ftctr254,./logs,...,210000,False,400000.0,1.0,NaN,0.0,0.0,single_random,1.0,NaN
94,0.002623,26.794468,0.949850,0.043470,249999,8,"SCaRF, finetuned (ours)",configs/282_blender_mic_8views_ftctr.txt,282_blender_mic_8views_ftctr254,./logs,...,250000,False,400000.0,1.0,NaN,0.0,0.0,single_random,1.0,NaN
93,0.002656,26.767822,0.950177,0.043008,299999,8,"SCaRF, finetuned (ours)",configs/282_blender_mic_8views_ftctr.txt,282_blender_mic_8views_ftctr254,./logs,...,300000,False,400000.0,1.0,NaN,0.0,0.0,single_random,1.0,NaN


./data/nerf_synthetic/ship


,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.reload_iter,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.hardcode_train_views,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp,args.num_render_poses
36,0.017642,17.692778,0.676209,0.290592,199999,4,SCaRF (ours),configs/245_blender_ship_4views_ctr.txt,245_blender_ship_4views_ctr,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,0.241816,6.332760,0.335781,0.593170,199999,4,NeRF,configs/225_blender_ship_4views.txt,225_blender_ship_4views,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,0.005805,22.835220,0.753288,0.205465,179999,8,SCaRF (ours),configs/255_blender_ship_8views_ctr.txt,255_blender_ship_8views_ctr,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.005507,23.043154,0.758032,0.204418,199999,8,"SCaRF, finetuned (ours)",configs/289_blender_ship_8views_ftctr.txt,289_blender_ship_8views_ftctr255,./logs,...,NaN,False,400000.0,1.0,NaN,0.0,0.0,single_random,1.0,NaN
14,0.030371,17.091694,0.694066,0.282706,199999,8,Simplified NeRF,configs/335_blender_ship_8views_simple.txt,335_blender_ship_8views_simple,./logs,...,None,False,200000.0,1.0,[],0.0,0.0,single_random,1.0,NaN
46,0.005507,23.043154,0.758032,0.204418,199999,8,SCaRF (ours),configs/255_blender_ship_8views_ctr.txt,255_blender_ship_8views_ctr,./logs,...,NaN,False,200000.0,10.0,NaN,0.0,0.0,single_random,1.0,NaN
57,0.097770,12.929134,0.605394,0.375103,199999,8,NeRF,configs/235_blender_ship_8views.txt,235_blender_ship_8views,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.005742,22.954083,0.762791,0.195434,209999,8,"SCaRF, finetuned (ours)",configs/289_blender_ship_8views_ftctr.txt,289_blender_ship_8views_ftctr255,./logs,...,210000,False,400000.0,1.0,NaN,0.0,0.0,single_random,1.0,NaN
6,0.006385,22.535612,0.756935,0.192647,249999,8,"SCaRF, finetuned (ours)",configs/289_blender_ship_8views_ftctr.txt,289_blender_ship_8views_ftctr255,./logs,...,250000,False,400000.0,1.0,NaN,0.0,0.0,single_random,1.0,NaN
3,0.006904,22.213859,0.751588,0.192393,299999,8,"SCaRF, finetuned (ours)",configs/289_blender_ship_8views_ftctr.txt,289_blender_ship_8views_ftctr255,./logs,...,300000,False,400000.0,1.0,NaN,0.0,0.0,single_random,1.0,NaN


# Group by scene, for last iteration

In [37]:
# Select evaluations of most recent training iteration
idx = metrics_df.groupby('args.config')['train_iters'].transform(max) == metrics_df['train_iters']
metrics_df_max_iters = metrics_df[idx]

In [38]:
# Print cleanly by dataset
for scene_key, scene_df in metrics_df_max_iters.groupby('args.datadir'):
    print(scene_key)
    scene_df = scene_df.sort_values([
        'views',
        'train_iters',
        'method'
    ])
    display(scene_df)

./data/nerf_synthetic/chair


,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.reload_iter,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.hardcode_train_views,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp,args.num_render_poses
51,0.052804,12.973991,0.601996,0.415411,199999,4,NeRF,configs/220_blender_chair_4views.txt,220_blender_chair_4views,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,0.017929,18.436800,0.799504,0.190794,199999,4,SCaRF (ours),configs/240_blender_chair_4views_ctr.txt,240_blender_chair_4views_ctr,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,0.011285,21.048566,0.860601,0.162509,199999,8,NeRF,configs/230_blender_chair_8views.txt,230_blender_chair_8views,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,0.004170,24.633213,0.898032,0.092438,199999,8,SCaRF (ours),configs/250_blender_chair_8views_ctr.txt,250_blender_chair_8views_ctr,./logs,...,NaN,False,200000.0,10.0,NaN,0.0,0.0,single_random,1.0,NaN
1,0.007454,21.869533,0.859454,0.151596,199999,8,Simplified NeRF,configs/330_blender_chair_8views_simple.txt,330_blender_chair_8views_simple,./logs,...,None,False,200000.0,1.0,[],0.0,0.0,single_random,1.0,NaN
84,0.003381,25.678945,0.914225,0.075666,399999,8,"SCaRF, finetuned (ours)",configs/283_blender_chair_8views_ftctr.txt,283_blender_chair_8views_ftctr250,./logs,...,NaN,False,400000.0,1.0,NaN,0.0,0.0,single_random,1.0,NaN
19,0.000412,34.073013,0.977810,0.024821,199999,100,NeRF,configs/chair.txt,blender_paper_chair,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


./data/nerf_synthetic/drums


,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.reload_iter,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.hardcode_train_views,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp,args.num_render_poses
34,0.086626,10.671803,0.590800,0.423424,199999,4,NeRF,configs/221_blender_drums_4views.txt,221_blender_drums_4views,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,0.045604,13.457292,0.675309,0.301415,199999,4,SCaRF (ours),configs/241_blender_drums_4views_ctr.txt,241_blender_drums_4views_ctr,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.018335,17.471725,0.769530,0.231024,199999,8,NeRF,configs/231_blender_drums_8views.txt,231_blender_drums_8views,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,0.010230,20.033635,0.843434,0.117413,199999,8,SCaRF (ours),configs/251_blender_drums_8views_ctr.txt,251_blender_drums_8views_ctr,./logs,...,NaN,False,200000.0,10.0,NaN,0.0,0.0,single_random,1.0,NaN
37,0.032963,15.020734,0.727292,0.280287,199999,8,Simplified NeRF,configs/331_blender_drums_8views_simple.txt,331_blender_drums_8views_simple,./logs,...,None,False,200000.0,1.0,[],0.0,0.0,single_random,1.0,NaN
70,0.010657,19.864415,0.842696,0.119325,399999,8,"SCaRF, finetuned (ours)",configs/284_blender_drums_8views_ftctr.txt,284_blender_drums_8views_ftctr251,./logs,...,NaN,False,400000.0,1.0,NaN,0.0,0.0,single_random,1.0,NaN
100,0.002971,25.529593,0.928950,0.063527,199999,100,NeRF,configs/drums.txt,blender_paper_drums,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


./data/nerf_synthetic/ficus


...

,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.reload_iter,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.hardcode_train_views,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp,args.num_render_poses
90,0.165159,7.979751,0.497654,0.468705,199999,4,NeRF,configs/227_blender_materials_4views.txt,226_blender_materials_4views,./logs,...,NaN,False,200000.0,1.0,NaN,0.0,0.0,single_random,1.0,NaN
52,0.210504,6.808063,0.394229,0.532183,199999,4,SCaRF (ours),configs/247_blender_materials_4views_ctr.txt,247_blender_materials_4views_ctr,./logs,...,NaN,False,200000.0,10.0,NaN,0.0,0.0,single_random,1.0,NaN
24,0.165473,7.837306,0.483915,0.466599,199999,8,NeRF,configs/237_blender_materials_8views.txt,237_blender_materials_8views,./logs,...,NaN,False,200000.0,1.0,NaN,0.0,0.0,single_random,1.0,NaN
13,0.008147,21.253789,0.842860,0.102170,199999,8,SCaRF (ours),configs/258_blender_materials_8views_ctr.txt,258_blender_materials_8views_ctr,./logs,...,NaN,False,200000.0,10.0,NaN,0.0,0.0,single_random,1.0,NaN
30,0.008937,20.659359,0.823098,0.150610,199999,8,Simplified NeRF,configs/347_blender_materials_8views_simpler.txt,347_blender_materials_8views_simpler,./logs,...,None,False,200000.0,1.0,[],0.0,0.0,single_random,1.0,NaN
27,0.007490,21.620932,0.850885,0.094869,249999,8,"SCaRF, finetuned (ours)",configs/287_blender_materials_8views_ftctr_200...,287_blender_materials_8views_ftctr258_200k,./logs,...,250000,False,400000.0,1.0,[],0.0,0.0,single_random,1.0,40.0
23,0.001220,29.339644,0.958222,0.037147,199999,100,NeRF,configs/materials.txt,materials,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


./data/nerf_synthetic/mic


,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.reload_iter,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.hardcode_train_views,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp,args.num_render_poses
87,0.058747,12.372348,0.707625,0.346917,199999,4,NeRF,configs/224_blender_mic_4views.txt,224_blender_mic_4views,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,0.012596,19.239916,0.877721,0.120375,199999,4,SCaRF (ours),configs/244_blender_mic_4views_ctr.txt,244_blender_mic_4views_ctr,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,0.003241,26.287436,0.944459,0.067203,199999,8,NeRF,configs/234_blender_mic_8views.txt,234_blender_mic_8views,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,0.002838,26.321344,0.943849,0.052638,199999,8,SCaRF (ours),configs/254_blender_mic_8views_ctr.txt,254_blender_mic_8views_ctr,./logs,...,NaN,False,200000.0,10.0,NaN,0.0,0.0,single_random,1.0,NaN
54,0.004553,24.206465,0.929758,0.080002,199999,8,Simplified NeRF,configs/334_blender_mic_8views_simple.txt,334_blender_mic_8views_simple,./logs,...,None,False,200000.0,1.0,[],0.0,0.0,single_random,1.0,NaN
91,0.002686,26.754484,0.950299,0.042610,399999,8,"SCaRF, finetuned (ours)",configs/282_blender_mic_8views_ftctr.txt,282_blender_mic_8views_ftctr254,./logs,...,NaN,False,400000.0,1.0,NaN,0.0,0.0,single_random,1.0,NaN
22,0.000494,33.196878,0.978615,0.022581,199999,100,NeRF,configs/mic.txt,blender_paper_mic,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


./data/nerf_synthetic/ship


,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.reload_iter,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.hardcode_train_views,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp,args.num_render_poses
74,0.241816,6.332760,0.335781,0.593170,199999,4,NeRF,configs/225_blender_ship_4views.txt,225_blender_ship_4views,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,0.017642,17.692778,0.676209,0.290592,199999,4,SCaRF (ours),configs/245_blender_ship_4views_ctr.txt,245_blender_ship_4views_ctr,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,0.097770,12.929134,0.605394,0.375103,199999,8,NeRF,configs/235_blender_ship_8views.txt,235_blender_ship_8views,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,0.005507,23.043154,0.758032,0.204418,199999,8,SCaRF (ours),configs/255_blender_ship_8views_ctr.txt,255_blender_ship_8views_ctr,./logs,...,NaN,False,200000.0,10.0,NaN,0.0,0.0,single_random,1.0,NaN
14,0.030371,17.091694,0.694066,0.282706,199999,8,Simplified NeRF,configs/335_blender_ship_8views_simple.txt,335_blender_ship_8views_simple,./logs,...,None,False,200000.0,1.0,[],0.0,0.0,single_random,1.0,NaN
4,0.007271,21.986228,0.746441,0.194135,399999,8,"SCaRF, finetuned (ours)",configs/289_blender_ship_8views_ftctr.txt,289_blender_ship_8views_ftctr255,./logs,...,NaN,False,400000.0,1.0,NaN,0.0,0.0,single_random,1.0,NaN
28,0.001215,29.407472,0.874514,0.124558,199999,100,NeRF,configs/ship.txt,blender_paper_ship,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Group by method, for last iteration, compute mean metrics

In [39]:
# Aggregate by method
grouped_df = metrics_df\
    .sort_values(['args.max_train_views', 'args.expname'])\
    .groupby(['views', 'method', 'train_iters'])

for k, v in grouped_df:
    mse = v['mse'].mean()
    psnr = v['psnr'].mean()
    ssim = v['ssim'].mean()
    lpips = v['lpips'].mean()
    print(f'{k[0]} views, {k[1]} mse={mse:.4f} psnr={psnr:.4f} ssim={ssim:.4f} lpips={lpips:.4f}')
    if len(v) != 8:
        print('!!WARN bad number of scenes')
    display(v)
    print('='*50)

4 views, NeRF mse=0.1076 psnr=10.8934 ssim=0.5841 lpips=0.4143


,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.reload_iter,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.hardcode_train_views,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp,args.num_render_poses
51,0.052804,12.973991,0.601996,0.415411,199999,4,NeRF,configs/220_blender_chair_4views.txt,220_blender_chair_4views,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,0.086626,10.671803,0.590800,0.423424,199999,4,NeRF,configs/221_blender_drums_4views.txt,221_blender_drums_4views,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0.018838,17.700611,0.827852,0.162645,199999,4,NeRF,configs/222_blender_ficus_4views.txt,222_blender_ficus_4views,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,0.132863,8.943158,0.491976,0.475585,199999,4,NeRF,configs/223_blender_lego_4views.txt,223_blender_lego_4views,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,0.058747,12.372348,0.707625,0.346917,199999,4,NeRF,configs/224_blender_mic_4views.txt,224_blender_mic_4views,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,0.241816,6.332760,0.335781,0.593170,199999,4,NeRF,configs/225_blender_ship_4views.txt,225_blender_ship_4views,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,0.103696,10.173140,0.619469,0.428852,199999,4,NeRF,configs/226_blender_hotdog_4views.txt,226_blender_hotdog_4views,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,0.165159,7.979751,0.497654,0.468705,199999,4,NeRF,configs/227_blender_materials_4views.txt,226_blender_materials_4views,./logs,...,NaN,False,200000.0,1.0,NaN,0.0,0.0,single_random,1.0,NaN


4 views, SCaRF (ours) mse=0.1823 psnr=7.5323 ssim=0.4183 lpips=0.5693
!!WARN bad number of scenes


,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.reload_iter,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.hardcode_train_views,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp,args.num_render_poses
38,0.166376,7.979202,0.426003,0.613955,29999,4,SCaRF (ours),configs/246_blender_hotdog_4views_ctr.txt,246_blender_hotdog_4views_ctr,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,0.198314,7.085420,0.410553,0.524728,29999,4,SCaRF (ours),configs/247_blender_materials_4views_ctr.txt,247_blender_materials_4views_ctr,./logs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


4 views, SCaRF (ours) mse=0.0835 psnr=13.2029 ssim=0.6352 lpips=0.3397


...

,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.reload_iter,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.hardcode_train_views,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp,args.num_render_poses
64,0.015154,20.752635,0.809918,0.156825,199999,100,SCaRF (ours),configs/314_blender_lego_8views_ctr_right.txt,314_blender_lego_8views_ctr_right,./logs,...,None,False,200000.0,10.0,"[2, 5, 8, 9, 10, 16, 34, 35, 40, 52, 53, 54, 5...",0.0,0.0,single_random,1.0,NaN
18,0.011339,20.763049,0.801088,0.168443,199999,100,SCaRF (ours),configs/319_blender_lego_ctr_right_tune.txt,319_blender_lego_ctr_right_tune,./logs,...,None,False,200000.0,1.0,"[2, 5, 8, 9, 10, 16, 34, 35, 40, 52, 53, 54, 5...",0.0,0.0,single_random,1.0,NaN


100 views, SCaRF, finetuned (ours) mse=0.0118 psnr=22.2112 ssim=0.8235 lpips=0.1426
!!WARN bad number of scenes


,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.reload_iter,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.hardcode_train_views,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp,args.num_render_poses
89,0.011828,22.211157,0.823528,0.142575,199999,100,"SCaRF, finetuned (ours)",configs/321_blender_lego_8views_right_ftctr314...,321_blender_lego_8views_right_ftctr314_130k,./logs,...,None,False,200000.0,10.0,"[2, 5, 8, 9, 10, 16, 34, 35, 40, 52, 53, 54, 5...",0.0,0.0,single_random,1.0,NaN


100 views, Simplified NeRF mse=0.0104 psnr=21.5528 ssim=0.8184 lpips=0.1605
!!WARN bad number of scenes


,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.reload_iter,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.hardcode_train_views,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp,args.num_render_poses
56,0.010389,21.552795,0.818427,0.160456,199999,100,Simplified NeRF,configs/317_blender_lego_8views_right_tune.txt,317_blender_lego_8views_right_tune,./logs,...,None,False,200000.0,1.0,"[2, 5, 8, 9, 10, 16, 34, 35, 40, 52, 53, 54, 5...",0.0,0.0,single_random,1.0,NaN


In [40]:
!ls /data/ajay/clip/nerf/nerf-pytorch/data/nerf_synthetic/chair/train -1 | wc -l

100
